In [10]:
#imports:
from collections import OrderedDict #to remove duplicates
import pandas as pd #dataframes
import numpy as np #np matrices
from sklearn import preprocessing #label encoding
from sklearn.feature_extraction.text import CountVectorizer #text transformation
from sklearn.naive_bayes import GaussianNB #naive Bayes classifier
from sklearn.model_selection import cross_val_score #cross validation scores

#define helper functions
def transformDate(x):
	return int(x.split('.')[2])*31*12+int(x.split('.')[1])*31+int(x.split('.')[0])

def getDay(x):
	return int(x.split('.')[0])

def getYear(x):
	return int(x.split('.')[2])

def getMonth(x):
	return int(x.split('.')[1])

In [11]:
#read in the csv file using pandas
#read file and create dataset, seperators are semicolons
dataset=pd.read_csv("ex1dataset.csv",sep=';',engine='python')
#drop the 'Unnamed: 0' column as this is only the line number
dataset.drop('Unnamed: 0',axis=1,inplace=True)
#explore dataset
print(dataset.head(5))

   Auftragskonto Buchungstag Valutadatum              Buchungstext  \
0     89990201.0  28.07.2016  28.07.2016             Lohn / Gehalt   
1     89990201.0  27.07.2016  27.07.2016                     Miete   
2     89990201.0  21.07.2016  21.07.2016                   Bargeld   
3     89990201.0  20.07.2016  20.07.2016  Lebensmittel / Getraenke   
4     89990201.0  18.07.2016  18.07.2016            Spontanausgabe   

                                    Verwendungszweck  \
0  Gehalt Adorsys GmbH & Co. KG End-To-End-Ref.: ...   
1  Byladem1Sbt De12773501123456789889 Miete Beuth...   
2  21.07/16.34Uhr Nuernberg All Eur 70,00 Geb.Eur...   
3  2831 Edeka Neubauer Nuernb.//Nuernb 2016-07-20...   
4                                             Amazon   

  Beguenstigter/Zahlungspflichtiger             Kontonummer          BLZ  \
0             Adorsys GmbH & Co. KG              7807800780     25190001   
1                      Georg Tasche  DE31251900019123456780  VOHADE2HXXX   
2             

In [12]:
#create a copy of the dataset for analysis
analysisSet=pd.read_csv("ex1dataset.csv",sep=';',engine='python')
analysisSet.drop('Unnamed: 0',axis=1,inplace=True)

#group set by label and print number of entries per label
print(analysisSet.groupby('label').agg('size'))

label
finance             33
income              17
leisure             65
living              26
private             21
standardOfLiving    47
dtype: int64


In [13]:
#analyse the impact of the features on the label

#convert 'Betrag' to float and replace commas by dots before
analysisSet['Betrag']=analysisSet['Betrag'].str.replace(',','.')
analysisSet['Betrag']=analysisSet['Betrag'].astype(float)

#fill NaN entries in 'Auftragskonto' by mean value of other columns
analysisSet['Auftragskonto']=analysisSet['Auftragskonto'].astype(float)
analysisSet['Auftragskonto'].fillna(analysisSet['Auftragskonto'].mean(),inplace=True)
analysisSet['Auftragskonto']=analysisSet['Auftragskonto'].astype(int)

#set all other columns as string entries
analysisSet['Kontonummer']=analysisSet['Kontonummer'].astype(str)
analysisSet['BLZ']=analysisSet['BLZ'].astype(str)
analysisSet['Buchungstag']=analysisSet['Buchungstag'].astype(str)
analysisSet['Valutadatum']=analysisSet['Valutadatum'].astype(str)

analysisSet['Day']=analysisSet['Buchungstag'].astype(str)
analysisSet['Day']=analysisSet['Day'].apply(lambda x: getDay(x))
analysisSet['Month']=analysisSet['Buchungstag'].astype(str)
analysisSet['Month']=analysisSet['Month'].apply(lambda x: getMonth(x))
analysisSet['Year']=analysisSet['Buchungstag'].astype(str)
analysisSet['Year']=analysisSet['Year'].apply(lambda x: getYear(x))

#transform Dates to numbers
analysisSet['Buchungstag']=dataset['Buchungstag'].apply(lambda x: transformDate(x))
analysisSet['Valutadatum']=dataset['Valutadatum'].apply(lambda x: transformDate(x))
analysisSet['Buchungstag']=analysisSet['Buchungstag'].astype(int)
analysisSet['Valutadatum']=analysisSet['Valutadatum'].astype(int)

#convert all string columns to numbers with the use of the label encoder
le=preprocessing.LabelEncoder()
for entry in ['Auftragskonto','Buchungstext',
'Verwendungszweck','Beguenstigter/Zahlungspflichtiger','Waehrung']:
	analysisSet[entry]=le.fit_transform(analysisSet[entry])
    
#group the set by labels and print the number of different values, the mean and the stddev for each column
print(analysisSet.groupby('label').agg(['mean','std','nunique']))

                 Auftragskonto                      Buchungstag             \
                          mean       std nunique           mean        std   
label                                                                        
finance               0.696970  0.728219       3  750084.666667  50.195036   
income                0.823529  0.808957       3  750087.882353  58.687608   
leisure               1.969231  0.248069       2  750037.784615  33.691844   
living                1.346154  0.845804       3  750068.153846  53.385910   
private               1.333333  0.856349       3  750076.904762  55.655103   
standardOfLiving      1.127660  0.849988       3  750094.085106  51.385512   

                            Valutadatum                    Buchungstext  ...  \
                 nunique           mean        std nunique         mean  ...   
label                                                                    ...   
finance               19  750084.666667  50.195036      1

In [14]:
#clean out the dataset based on the revalations of the analysis

#keep day for use later on as maybe there might be covariance between day and 
dataset['Day']=dataset['Buchungstag'].apply(lambda x: getDay(x))
dataset['Day']=dataset['Day'].astype(int)
#Waehrung: Analysis showed only one unique value so there is no gain in keeping it for classification
#Kontonummer,Auftagskonto,BLZ: numbers that were assigned to the associated accounts likely not correlate to the kind of transaction.
#(Analysis also showed little to no difference between the different labels)
#Buchungstag,Valutdatum: assumption that the date of a purchase does not correlate with the class of transaction we are trying to predict
#(The analysis does not quite support that assumption but I'm still holding on to that assumption :D)
dataset.drop(['Waehrung','Kontonummer','Auftragskonto','Buchungstag','Valutadatum','BLZ'],axis=1,inplace=True)
#explore the reduced dataset
print(dataset.head(5))

               Buchungstext  \
0             Lohn / Gehalt   
1                     Miete   
2                   Bargeld   
3  Lebensmittel / Getraenke   
4            Spontanausgabe   

                                    Verwendungszweck  \
0  Gehalt Adorsys GmbH & Co. KG End-To-End-Ref.: ...   
1  Byladem1Sbt De12773501123456789889 Miete Beuth...   
2  21.07/16.34Uhr Nuernberg All Eur 70,00 Geb.Eur...   
3  2831 Edeka Neubauer Nuernb.//Nuernb 2016-07-20...   
4                                             Amazon   

  Beguenstigter/Zahlungspflichtiger   Betrag             label  Day  
0             Adorsys GmbH & Co. KG  2000.00            income   28  
1                      Georg Tasche  -670.00            living   27  
2                           Bargeld   -70.00           private   21  
3                     Kartenzahlung   -73.21  standardOfLiving   20  
4                  neue Playstation     -363           leisure   18  


In [15]:
#treat the data to make it suitable for a skit.learn classifier

#remove special characters from 'Buchungstext','Vetwendungszweck' and 'Beguenstigter/Zahlungspflichtuger'
for label in ['Buchungstext','Verwendungszweck','Beguenstigter/Zahlungspflichtiger']:
	dataset[label]=dataset[label].str.replace('/',' ')
	dataset[label]=dataset[label].str.replace('(',' ')
	dataset[label]=dataset[label].str.replace(')',' ')
	dataset[label]=dataset[label].str.replace(':',' ')
	dataset[label]=dataset[label].str.replace('.',' ')
	dataset[label]=dataset[label].str.replace('&',' ')
	dataset[label]=dataset[label].str.replace(',',' ')
    
    
#join the three colomns 'Buchungstext','Verwendungszweck' and 'Beguenstigter/Zahlungspflichtiger'
#(this is to combat the fact that these columns are intermixed a lot depending on the data source)
dataset['Text']=dataset[['Buchungstext','Verwendungszweck','Beguenstigter/Zahlungspflichtiger']].agg(" ".join,axis=1)
#drop the original columns
dataset.drop(['Buchungstext','Verwendungszweck','Beguenstigter/Zahlungspflichtiger'],axis=1,inplace=True)

#remove all entries in 'Text' that contain numbers as they likely belong to information that is not necessary for prediction (Times, Account numbers, Dates, Postal Codes) or redundant Values (Transaction values) and only affect few useful pieces
dataset['Text']=dataset['Text'].str.replace(r'\w*\d\w*','',regex=True)
#remove duplicate words in 'Text'
dataset['Text']=dataset['Text'].str.split().apply(lambda x: OrderedDict.fromkeys(x).keys()).str.join(' ')

#replace all commas in 'Betrag' by dots to keep the encoding consistent
dataset['Betrag']=dataset['Betrag'].str.replace(',','.')
#convert 'Betrag' entries to numerical values
dataset['Betrag']=dataset['Betrag'].astype(float)
#convert labels to numerical data
le=preprocessing.LabelEncoder()
dataset['label']=le.fit_transform(dataset['label'])

#explore the treated dataset
print(dataset.head(5))

    Betrag  label  Day                                               Text
0  2000.00      1   28  Lohn Gehalt Adorsys GmbH Co KG End-To-End-Ref ...
1  -670.00      3   27  Miete Beuthener Str End-To-End-Ref Notprovided...
2   -70.00      4   21  Bargeld Nuernberg All Eur Geb Einzahlung Ausza...
3   -73.21      5   20  Lebensmittel Getraenke Edeka Neubauer Nuernb -...
4  -363.00      2   18             Spontanausgabe Amazon neue Playstation


In [16]:
#encode the string feature to a bag of words using CountVectorizer
cv=CountVectorizer()
cv.fit(dataset['Text'])
Text_tranformed=cv.transform(dataset['Text']).todense()
#split labels from dataset and keep colume as Y values and Rest as X values
Y_vals=dataset['label']
X_vals=np.hstack((dataset['Betrag'].values.reshape(len(dataset['Betrag']),1),Text_tranformed))

print(X_vals)
print(Y_vals)

[[ 2.00e+03  0.00e+00  0.00e+00 ...  0.00e+00  0.00e+00  0.00e+00]
 [-6.70e+02  0.00e+00  0.00e+00 ...  0.00e+00  0.00e+00  0.00e+00]
 [-7.00e+01  0.00e+00  0.00e+00 ...  0.00e+00  0.00e+00  0.00e+00]
 ...
 [ 2.00e+03  0.00e+00  0.00e+00 ...  0.00e+00  0.00e+00  0.00e+00]
 [-4.00e+03  0.00e+00  0.00e+00 ...  0.00e+00  0.00e+00  0.00e+00]
 [-5.12e+01  1.00e+00  0.00e+00 ...  0.00e+00  0.00e+00  0.00e+00]]
0      1
1      3
2      4
3      5
4      2
      ..
204    0
205    0
206    1
207    0
208    2
Name: label, Length: 209, dtype: int64


In [17]:
#lets add day to the feature matrix
X_vals=np.hstack((dataset['Day'].values.reshape(len(dataset['Day']),1),X_vals))
print(X_vals)

[[ 2.80e+01  2.00e+03  0.00e+00 ...  0.00e+00  0.00e+00  0.00e+00]
 [ 2.70e+01 -6.70e+02  0.00e+00 ...  0.00e+00  0.00e+00  0.00e+00]
 [ 2.10e+01 -7.00e+01  0.00e+00 ...  0.00e+00  0.00e+00  0.00e+00]
 ...
 [ 2.80e+01  2.00e+03  0.00e+00 ...  0.00e+00  0.00e+00  0.00e+00]
 [ 2.80e+01 -4.00e+03  0.00e+00 ...  0.00e+00  0.00e+00  0.00e+00]
 [ 8.00e+00 -5.12e+01  1.00e+00 ...  0.00e+00  0.00e+00  0.00e+00]]


In [18]:
#train and score a gaussian naive bayes classifier
gnb= GaussianNB()
scores=cross_val_score(gnb,X_vals,Y_vals,cv=5)
print("mean: "+str(scores.mean())+" var: "+str(scores.std()))

mean: 0.8902439024390244 var: 0.08035370016766939
